# Computing temperature properties and mixing length for the SR1b section data, considering the full section (also the part that was not sampled by the DIMES mooring)

In [1]:
import pickle
import h5py
import numpy as np
import xarray as xr
from scipy.io import loadmat
import gsw
import csaps
import warnings

## Read in data, compute SA, CT, $\phi_{1500}$

In [2]:
#%% READ IN HYDROGRAPHIC SECTION DATA
data = loadmat('data/SR1b_section/sr1b_all.mat')

icyc = np.where(data['buf_year'] != 1999)[1] # exclude 1999

buf_lat = data['buf_lat'][0,icyc]
buf_lon = data['buf_lon'][0,icyc]
buf_sal = data['buf_sal'][:,icyc]
buf_temp = data['buf_temp'][:,icyc]
buf_ptmp = data['buf_ptmp'][:,icyc]
buf_press = data['buf_press'].reshape(3000)
buf_year = data['buf_year'][:,icyc].reshape(316)

f = h5py.File('data/SR1b_section/sr1b_gamma.mat')
buf_gamma = f["gamma"][icyc,:].transpose()

In [3]:
# sample points along section
start_lon_vals = []
end_lon_vals = []
start_lat_vals = []
end_lat_vals = []
for yr in np.unique(buf_year):
  idx_yr = np.where(buf_year == yr)[0]
  start_lon_vals.append(np.min(buf_lon[idx_yr]))
  end_lon_vals.append(np.max(buf_lon[idx_yr]))
  start_lat_vals.append(np.max(buf_lat[idx_yr]))
  end_lat_vals.append(np.min(buf_lat[idx_yr]))
start_lon = np.mean(np.array(start_lon_vals))
end_lon = np.mean(np.array(end_lon_vals))
start_lat = np.mean(np.array(start_lat_vals))
end_lat = np.mean(np.array(end_lat_vals))

# Extract unique points along the section
nr_points = 30
lon_along_section = np.linspace(start_lon, end_lon, nr_points)
lat_along_section = np.linspace(start_lat, end_lat, nr_points)

# calculate distance from southern end of section
dist_along_section = np.zeros(len(lat_along_section))
for i in range(len(dist_along_section)):
  dist_along_section[i] = gsw.distance(np.mean(lon_along_section)*np.ones(2), np.array([min(lat_along_section), lat_along_section[i]]))[0]

# fit smoothing spline through relationship between distance and latitude/longitude along section
pp_dist_lat = csaps.csaps(dist_along_section[::-1], lat_along_section[::-1], smooth=0.85)
pp_lat_lon = csaps.csaps(lat_along_section[::-1], lon_along_section[::-1], smooth=0.85)
# save the spline fit
with open('data/pp_dist_lat.pkl', 'wb') as f:
    pickle.dump(pp_dist_lat, f)
with open('data/pp_lat_lon.pkl', 'wb') as f:
    pickle.dump(pp_lat_lon, f)

/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/csaps/_sspumv.py:301: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = la.spsolve(a, b)


In [4]:
# calculate absolute salinity and conservative temperature
buf_SA = gsw.SA_from_SP(buf_sal, buf_press.reshape(3000,1), buf_lon.reshape(1,316), buf_lat.reshape(1,316)) # absolute salinity
buf_CT = gsw.conversions.CT_from_t(buf_SA, buf_temp, buf_press.reshape(3000,1)) # conservative temperature

# compute baroclinic streamfunction
gpan_buf = -gsw.geo_strf_dyn_height(buf_SA, buf_CT, buf_press, 1500) #geopotential height anomaly / dynamic height anomalyß
buf_phi1500 = abs(gpan_buf[249,:]) # index 250 is at 500 dbar

# calculate distance from southern end of section
buf_dist = np.zeros(len(buf_lat))
for i in range(len(buf_lat)):
  buf_dist[i] = gsw.distance(np.mean(buf_lon)*np.ones(2), np.array([min(buf_lat), buf_lat[i]]))[0]

# fit a smoothing spline through relationship
pp_dist = csaps.csaps(buf_phi1500[np.argsort(buf_phi1500)[:-np.count_nonzero(np.isnan(buf_phi1500))]],
                      buf_dist[np.argsort(buf_phi1500)[:-np.count_nonzero(np.isnan(buf_phi1500))]],
                      smooth = 0.85)

# apply fitted spline for current data
distfromphi = pp_dist(buf_phi1500)

## Linear interpolation of variables on desired neutral surfaces

In [8]:
win_width = 0.02
igamma = np.arange(26.5, 28.34, win_width)
surf_press = np.zeros((len(igamma),len(buf_lat)))
surf_CT = np.zeros((len(igamma),len(buf_lat)))
surf_SA = np.zeros((len(igamma),len(buf_lat)))

for j in range(len(buf_lat)):
    with warnings.catch_warnings(): # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
        for i in range(len(igamma)):
            icyc = np.where((buf_gamma[:,j] <= igamma[i] + win_width/2) & (buf_gamma[:,j] >= igamma[i] - win_width/2))[0]
            surf_press[i,j] = np.nanmean(buf_press[icyc])
            surf_CT[i,j] = np.nanmean(buf_CT[icyc,j])
            surf_SA[i,j] = np.nanmean(buf_SA[icyc,j])

In [7]:
np.nanmin(buf_gamma), np.nanmax(buf_gamma)

(np.float64(26.487517651803728), np.float64(28.340706695653807))

## Calculate mean and rms of temperature and salinity plus their gradients on each neutral surface

In [10]:
# fit splines on isoneutral surfaces to determine mean relationships
pp_CT = [[] for i in range(len(igamma))]  # initialize array to fill with spline fits
pp_SA = [[] for i in range(len(igamma))]

for j in range(len(igamma)):  # only rows where there are actually non-NaNs, see surf_CT[77,:][~np.isnan(surf_CT[77,:])]
    # fit a smoothing spline through relationship
    idx_x = np.argsort(distfromphi)[:-np.count_nonzero(np.isnan(distfromphi))] # non-nan indices in x (for spline fit)
    idx_y = np.argwhere(~np.isnan(surf_CT[j,:])) #non-nan indices in y      

    new_idx = []
    for i in range(len(idx_x)):
        if idx_x[i] in idx_y:
            new_idx.append(idx_x[i])
    new_idx = np.asarray(new_idx)

    # now do spline fit
    smooth = 1e-14
    pp_CT[j] = csaps.csaps(distfromphi[new_idx], surf_CT[j,:][new_idx], smooth=smooth)
    pp_SA[j] = csaps.csaps(distfromphi[new_idx], surf_SA[j,:][new_idx], smooth=smooth)

In [12]:
# calculate "background" meridional theta gradient and rms theta anomaly for each isopycnal using a running meridional window

# define a meridional distance grid, and the width of the running window
idist = np.arange(0,680e3,10e3)
win_width = 40e3

pressmean     = np.zeros((len(igamma), len(idist)))
SAbar         = np.zeros((len(igamma), len(idist)))
thetabar      = np.zeros((len(igamma), len(idist)))
dthetabardy   = np.zeros((len(igamma), len(idist)))
rms_theta     = np.zeros((len(igamma), len(idist)))

for j in range(len(igamma)):
    dthetabardy[j,:] = np.gradient(pp_CT[j](idist),idist)
    with warnings.catch_warnings(): # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
        for i in range(len(idist)):
            icyc = np.where((distfromphi <= idist[i] + win_width/2) & (distfromphi >= idist[i] - win_width/2))[0]

            pressmean[j,i] = np.nanmean(surf_press[j,icyc])
            thetabar[j,i] = pp_CT[j](idist[i])
            SAbar[j,i]    = pp_SA[j](idist[i])

            if len(icyc) > 1:
                rms_theta[j,i] = np.nanstd(surf_CT[j,icyc] - pp_CT[j](distfromphi[icyc]))
            else:
                rms_theta[j,i] = np.nan

SAbar[np.isnan(pressmean)] = np.nan
thetabar[np.isnan(pressmean)] = np.nan
dthetabardy[np.isnan(pressmean)] = np.nan
rms_theta[np.isnan(pressmean)] = np.nan

## Calculate mixing length

In [13]:
# mixing efficiency parameter
gamma_mix = 0.35

# based on temperature
dthetabardy_limited = dthetabardy.copy()
dthetabardy_limited[np.where(abs(dthetabardy_limited) < 1e-7)] = 1e-7
Lmix = rms_theta / np.abs(dthetabardy_limited) 
Lmix[np.where(Lmix == 0)] = np.nan

## Saving the output as a function of cross-stream distance and neutral density

In [ ]:
ds = xr.Dataset({"pressmean": (["gamma", "dist"], pressmean),
                 "thetabar": (["gamma", "dist"], thetabar),
                 "dthetabardy": (["gamma", "dist"], dthetabardy_limited),
                 "rms_theta": (["gamma", "dist"], rms_theta),
                 "SAbar": (["gamma", "dist"], SAbar),
                 "Lmix": (["gamma", "dist"], Lmix)},
                coords={"gamma": igamma, "dist": idist})
ds.to_netcdf('data/variables_from_section-dist_gamma-full.nc')

## Now transform to more intuitive coordinates: latitude and depth

In [15]:
# transform cross-stream distance to latitude
ilat = pp_dist_lat(idist)  # latitude values corresponding to idist

# compute depth from pressure, for both moorings and section data
zmean = np.zeros(np.shape(pressmean))
for i in range(np.shape(pressmean)[1]):
    zmean[:,i] = gsw.z_from_p(pressmean[:,i], pp_dist_lat(idist[i]))*-1

In [16]:
# interpolate data from (Y, gamma) to (Y, depth) coordinates (gamma-levels to z-levels)
def convert_to_zlevs(data):
    z_levels = np.linspace(np.nanmin(zmean), np.nanmax(zmean), 60)
    Z = len(z_levels)
    L = len(idist)

    data_on_press = np.full((Z, L), np.nan)
    for i in range(L):
        z_col = zmean[:,i]
        d_col = data[:,i]

        # mask NaNs to avoid interpolation errors
        valid = np.isfinite(z_col) & np.isfinite(d_col)
        if np.count_nonzero(valid) < 2:
            continue  # Not enough points to interpolate

        # if pressure is not monotonic, sort it
        z_sorted = z_col[valid]
        d_sorted = d_col[valid]
        sort_idx = np.argsort(z_sorted)
        z_sorted = z_sorted[sort_idx]
        d_sorted = d_sorted[sort_idx]

        # interpolate to the pressure levels
        data_on_press[:, i] = np.interp(z_levels, z_sorted, d_sorted, left=np.nan, right=np.nan)
    return data_on_press, z_levels

# convert variables to z levels
thetabar_z, iz = convert_to_zlevs(thetabar)
dthetabardy_z, _ = convert_to_zlevs(dthetabardy)
dthetabardy_limited_z, _ = convert_to_zlevs(dthetabardy_limited)
rms_theta_z, _ = convert_to_zlevs(rms_theta)
SAbar_z, _ = convert_to_zlevs(SAbar)
Lmix_z, _ = convert_to_zlevs(Lmix)

dthetabardy_limited_z[np.where(abs(dthetabardy_limited_z) < 1e-7)] = 1e-7

In [17]:
ds = xr.Dataset({"thetabar": (["depth", "lat"], thetabar_z),
                 "dthetabardy": (["depth", "lat"], dthetabardy_limited_z),
                 "rms_theta": (["depth", "lat"], rms_theta_z),
                 "SAbar": (["depth", "lat"], SAbar_z),
                 "Lmix": (["depth", "lat"], Lmix_z)},
                coords={"depth": iz, "lat": ilat})
ds.to_netcdf('data/variables_from_section-lat_depth-full.nc')